Challenge Exercise

-----Download the CSV dataset of the StackOverflow 2019 Survey:-------

Using Spark SQL, create DataFrames, which filter for the following:
- Average age by gender.
- Top five countries by number of respondents.
- The oldest age to write the first line of code or program for United States respondents with a computer science, computer engineering, software engineering, mathematics, or statistics degree.


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [3]:
# 2. Initializing SparkSession

spark = SparkSession.builder.appName('gender_survey').getOrCreate()

In [4]:
# 3. Read CSV File with Spark DataFrame API

df_survey_2019 = spark.read.load(r"C:\Users\12242\Downloads\stack-overflow-developer-survey-2019\survey_results_public.csv",\
     format="csv", header = True,inferSchema = True)

df_survey_2019.printSchema()

root
 |-- Respondent: integer (nullable = true)
 |-- MainBranch: string (nullable = true)
 |-- Hobbyist: string (nullable = true)
 |-- OpenSourcer: string (nullable = true)
 |-- OpenSource: string (nullable = true)
 |-- Employment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Student: string (nullable = true)
 |-- EdLevel: string (nullable = true)
 |-- UndergradMajor: string (nullable = true)
 |-- EduOther: string (nullable = true)
 |-- OrgSize: string (nullable = true)
 |-- DevType: string (nullable = true)
 |-- YearsCode: string (nullable = true)
 |-- Age1stCode: string (nullable = true)
 |-- YearsCodePro: string (nullable = true)
 |-- CareerSat: string (nullable = true)
 |-- JobSat: string (nullable = true)
 |-- MgrIdiot: string (nullable = true)
 |-- MgrMoney: string (nullable = true)
 |-- MgrWant: string (nullable = true)
 |-- JobSeek: string (nullable = true)
 |-- LastHireDate: string (nullable = true)
 |-- LastInt: string (nullable = true)
 |-- FizzBuzz: 

In [5]:
# change the data type to integer

df_survey_2019= df_survey_2019.withColumn("Age", df_survey_2019.Age.cast(IntegerType()))

In [6]:
# - Average age by gender.

df_survey_2019.select("gender","age").groupBy("gender").avg().show()


+--------------------+------------------+
|              gender|          avg(age)|
+--------------------+------------------+
|                 Man|30.424181874074282|
|                  NA| 30.35640785781104|
|Man;Non-binary, g...|28.908536585365855|
|           Woman;Man| 26.23170731707317|
|Woman;Man;Non-bin...| 29.78723404255319|
|Woman;Non-binary,...|28.210884353741495|
|Non-binary, gende...|29.018036072144287|
|               Woman| 29.49538679914833|
+--------------------+------------------+



In [7]:
# - Top five countries by number of respondents.
df2 = df_survey_2019.groupBy('Country').count().orderBy(desc('count')).show(5)
## df2.orderBy(desc('count')).show(5)

#df_survey_2019.select("Country", "Respondent").groupby("Country").agg(count("Respondent"). alias("Respondent_countrywise")).sort(desc("Respondent_countrywise")).show(5)

+--------------+----------------------+
|       Country|Respondent_countrywise|
+--------------+----------------------+
| United States|                 20949|
|         India|                  9061|
|       Germany|                  5866|
|United Kingdom|                  5737|
|        Canada|                  3395|
+--------------+----------------------+
only showing top 5 rows



In [9]:
# - The oldest age to write the first line of code or program for United States respondents with a computer science, 
# computer engineering, software engineering, mathematics, or statistics degree.

# change to integer
df_survey_2019 = df_survey_2019.withColumn("Age1stCode", df_survey_2019.Age1stCode.cast(IntegerType()))

In [ ]:
df.select("Country", "UndergradMajor", "Age1stCode").filter

In [10]:
df_survey_2019.select("Country", "UndergradMajor", "Age1stCode").filter((df_survey_2019['Country'] == 'United States')\
      & ((df_survey_2019["UndergradMajor"] == "Computer science, computer engineering, or software engineering")\
         |(df_survey_2019["UndergradMajor"] == "Mathematics or statistics"))) \
    .groupby("Country", "UndergradMajor").max("Age1stCode").show()

+-------------+--------------------+---------------+
|      Country|      UndergradMajor|max(Age1stCode)|
+-------------+--------------------+---------------+
|United States|Computer science,...|             54|
|United States|Mathematics or st...|             56|
+-------------+--------------------+---------------+



spark.sql("SELECT MAX(Age1stCode) as oldest_age FROM `surveys`  \ WHERE Country = 'United States' AND (UndergradMajor = 'Computer science, computer engineering, or software engineering' OR UndergradMajor = 'Mathematics or statistics') AND Age IS NOT NULL").show()